In [3]:
import pandas as pd
import csv
import numpy as np
import time
import os
timestr = time.strftime("%m%d%Y_%H%m%s")

In [4]:
pth = '/ctao_shared_data/AHA/jfeng_VET/'
os.listdir(pth)

['bleeding.types',
 'bleeding copy.types',
 'filtered_history_label_columns_98236_VET_3y_event_label.csv']

In [18]:
history = pd.read_csv(pth+'filtered_history_label_columns_98236_VET_3y_event_label.csv',low_memory=False)

In [19]:
history.columns

Index(['patient_sk', 'age', 'gender', 'race', 'gender_label', 'race_label',
       'age_group_label', 'prior_bleeding_label', 'bleeding_label',
       'intracranial_bleeding_label', 'GI_bleeding_label',
       'transfusion_code_label', 'revascularization_label',
       'stenting_vein_graft_label', 'alcoholo_abuse_label', 'anemia_label',
       'atrial_fibrillation_label', 'cancer_label', 'skin_cancer_label',
       'CKD_label', 'diabetes_label', 'dyslipidemia_code_label',
       'heart_failure_label', 'HBP_label', 'acute_ischemic_event_label',
       'liver_disease_label', 'PAD_label', 'prior_smoke_label',
       'smoke_disease_label', 'smoke_disease_procedure_label', 'stroke_label',
       'ACE_inhibitors_label', 'angiotensin_receptor_blockers_label',
       'beta_blockers_label', 'calcium_antagonists_label', 'NSAID_label',
       'statins_label', 'bleeding_3y_event_label', 'ischemic_3y_event_label',
       'VTE'],
      dtype='object')

In [20]:
history.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98236 entries, 0 to 98235
Data columns (total 40 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   patient_sk                           98236 non-null  int64  
 1   age                                  98236 non-null  float64
 2   gender                               98234 non-null  object 
 3   race                                 98090 non-null  object 
 4   gender_label                         98234 non-null  float64
 5   race_label                           98090 non-null  float64
 6   age_group_label                      98236 non-null  int64  
 7   prior_bleeding_label                 98236 non-null  int64  
 8   bleeding_label                       98236 non-null  int64  
 9   intracranial_bleeding_label          98236 non-null  int64  
 10  GI_bleeding_label                    98236 non-null  int64  
 11  transfusion_code_label      

## Group Race to Other Race

In [21]:
history.race.value_counts()

Caucasian                 81518
African American           9532
Other                      3533
Asian                      1466
Native American             809
Hispanic                    590
Unknown                     393
Pacific Islander            169
Mid Eastern Indian           34
Biracial                     31
Asian/Pacific Islander       15
Name: race, dtype: int64

In [22]:
oth_ls = ['Native American', 'Pacific Islander', 'Asian/Pacific Islander','Biracial', 'Mid Eastern Indian','Other','Unknown']
history['Race'] = np.where(history.race.isin(oth_ls),'Other',history.race)
history.Race.value_counts()

Caucasian           81518
African American     9532
Other                4984
Asian                1466
Hispanic              590
Name: Race, dtype: int64

In [23]:
history['patient_sk']=history['patient_sk'].astype(str)
history['age'] = history['age'].astype(int)

In [24]:

history.race_label.value_counts()

0.0    81518
1.0     9532
5.0     3782
3.0     1466
2.0      809
4.0      590
6.0      393
Name: race_label, dtype: int64

In [25]:
history = history.drop(columns=['gender_label','race_label','race']).reset_index(drop=True)

In [26]:
history = pd.get_dummies(data = history, columns=['Race','gender'])

In [28]:
history.columns

Index(['patient_sk', 'age', 'age_group_label', 'prior_bleeding_label',
       'bleeding_label', 'intracranial_bleeding_label', 'GI_bleeding_label',
       'transfusion_code_label', 'revascularization_label',
       'stenting_vein_graft_label', 'alcoholo_abuse_label', 'anemia_label',
       'atrial_fibrillation_label', 'cancer_label', 'skin_cancer_label',
       'CKD_label', 'diabetes_label', 'dyslipidemia_code_label',
       'heart_failure_label', 'HBP_label', 'acute_ischemic_event_label',
       'liver_disease_label', 'PAD_label', 'prior_smoke_label',
       'smoke_disease_label', 'smoke_disease_procedure_label', 'stroke_label',
       'ACE_inhibitors_label', 'angiotensin_receptor_blockers_label',
       'beta_blockers_label', 'calcium_antagonists_label', 'NSAID_label',
       'statins_label', 'bleeding_3y_event_label', 'ischemic_3y_event_label',
       'VTE', 'Race_African American', 'Race_Asian', 'Race_Caucasian',
       'Race_Hispanic', 'Race_Other', 'gender_Female', 'gender_Male',

In [29]:
history.shape

(98236, 44)

In [30]:
isch_no = history.loc[history['ischemic_3y_event_label']==0]
isch_yes = history.loc[history['ischemic_3y_event_label']==1]
bld_no = history.loc[history['bleeding_3y_event_label']==0]
bld_yes = history.loc[history['bleeding_3y_event_label']==1]
al_d = {'isch_no':isch_no}
isch_yes_d = {'isch_yes':isch_yes}
bld_no_d = {'bld_no':bld_no}
bld_yes_d = {'bld_yes':bld_yes}

In [31]:
al_d.update(isch_yes_d)
al_d.update(bld_no_d)
al_d.update(bld_yes_d)

In [32]:
al_d.keys()

dict_keys(['isch_no', 'isch_yes', 'bld_no', 'bld_yes'])

In [33]:
count_dict={}
d={}
    
for key in al_d.keys():
    data=pd.DataFrame(al_d[key])
    count_dict[key]={}
    # Age, mean and median
    d['age_mean'] = data['age'].mean()
    d['age_std'] = data['age'].std()
    
    
    
    # Calculate the number of ==1 for each label in the column, and the total count of all patients
    col = list(data.columns)
    for x in col:
        yes = len(data.loc[data[x]==1])
        d[x]=yes
        total = len(data)
        d['total']=total
    
    count_dict[key].update(d)
    
print(count_dict)

{'isch_no': {'age_mean': 64.50349067716265, 'age_std': 12.003262029018238, 'patient_sk': 0, 'total': 69614, 'age': 0, 'age_group_label': 30541, 'prior_bleeding_label': 5460, 'bleeding_label': 4421, 'intracranial_bleeding_label': 156, 'GI_bleeding_label': 1926, 'transfusion_code_label': 1594, 'revascularization_label': 1162, 'stenting_vein_graft_label': 0, 'alcoholo_abuse_label': 849, 'anemia_label': 7533, 'atrial_fibrillation_label': 4785, 'cancer_label': 5913, 'skin_cancer_label': 839, 'CKD_label': 5183, 'diabetes_label': 2373, 'dyslipidemia_code_label': 26450, 'heart_failure_label': 7392, 'HBP_label': 29818, 'acute_ischemic_event_label': 7329, 'liver_disease_label': 16078, 'PAD_label': 9007, 'prior_smoke_label': 10670, 'smoke_disease_label': 10670, 'smoke_disease_procedure_label': 0, 'stroke_label': 2766, 'ACE_inhibitors_label': 9340, 'angiotensin_receptor_blockers_label': 3594, 'beta_blockers_label': 15816, 'calcium_antagonists_label': 8424, 'NSAID_label': 7950, 'statins_label': 131

In [34]:
r={}
    
for key in count_dict.keys():
    r[key]={}
    total = count_dict[key]['total']
    for name,value in count_dict[key].items():
        dd={}
        dd[name]= value/total
        r[key].update(dd)

In [35]:
count_dict.keys()

dict_keys(['isch_no', 'isch_yes', 'bld_no', 'bld_yes'])

In [36]:
count_df = pd.DataFrame.from_dict(count_dict)
r_df = pd.DataFrame.from_dict(r)

In [37]:
count_df.head()

,isch_no,isch_yes,bld_no,bld_yes
age_mean,64.503491,64.165537,63.825280,67.336704
age_std,12.003262,12.200662,11.988329,12.007069
patient_sk,0.000000,0.000000,0.000000,0.000000
total,69614.000000,28622.000000,82017.000000,16219.000000
age,0.000000,0.000000,0.000000,0.000000


In [38]:
r_df.head()

,isch_no,isch_yes,bld_no,bld_yes
age_mean,0.000927,0.002242,0.000778,0.004152
age_std,0.000172,0.000426,0.000146,0.000740
patient_sk,0.000000,0.000000,0.000000,0.000000
total,1.000000,1.000000,1.000000,1.000000
age,0.000000,0.000000,0.000000,0.000000


In [39]:
count_df['isch_no_r']=r_df['isch_no']
count_df['isch_yes_r']=r_df['isch_yes']
count_df['bld_no_r']=r_df['bld_no']
count_df['bld_yes_r']=r_df['bld_yes']

In [40]:
rs_df=count_df[['isch_yes','isch_yes_r','isch_no','isch_no_r','bld_yes','bld_yes_r','bld_no','bld_no_r']]

In [41]:
rs_df

,isch_yes,isch_yes_r,isch_no,isch_no_r,bld_yes,bld_yes_r,bld_no,bld_no_r
age_mean,64.165537,0.002242,64.503491,0.000927,67.336704,0.004152,63.825280,0.000778
age_std,12.200662,0.000426,12.003262,0.000172,12.007069,0.000740,11.988329,0.000146
patient_sk,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
total,28622.000000,1.000000,69614.000000,1.000000,16219.000000,1.000000,82017.000000,1.000000
age,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
age_group_label,12859.000000,0.449270,30541.000000,0.438719,5688.000000,0.350700,37712.000000,0.459807
prior_bleeding_label,2862.000000,0.099993,5460.000000,0.078432,2892.000000,0.178309,5430.000000,0.066206
bleeding_label,2275.000000,0.079484,4421.000000,0.063507,2318.000000,0.142919,4378.000000,0.053379
intracranial_bleeding_label,106.000000,0.003703,156.000000,0.002241,73.000000,0.004501,189.000000,0.002304
GI_bleeding_label,1051.000000,0.036720,1926.000000,0.027667,1109.000000,0.068377,1868.000000,0.022776


In [42]:
rs_df.style.format({'isch_no': "{:.0f}", 'isch_no_r': '{0:.1}',
                   'isch_yes': "{:.0f}", 'isch_yes_r': '{0:.1}',
                   'bld_no': "{:.0f}", 'bld_no_r': '{0:.1}',
                   'bld_yes': "{:.0f}", 'bld_yes_r': '{0:.1}'})

,isch_yes,isch_yes_r,isch_no,isch_no_r,bld_yes,bld_yes_r,bld_no,bld_no_r
age_mean,64,0.002,65,0.0009,67,0.004,64,0.0008
age_std,12,0.0004,12,0.0002,12,0.0007,12,0.0001
patient_sk,0,0e+00,0,0e+00,0,0e+00,0,0e+00
total,28622,1e+00,69614,1e+00,16219,1e+00,82017,1e+00
age,0,0e+00,0,0e+00,0,0e+00,0,0e+00
age_group_label,12859,0.4,30541,0.4,5688,0.4,37712,0.5
prior_bleeding_label,2862,0.1,5460,0.08,2892,0.2,5430,0.07
bleeding_label,2275,0.08,4421,0.06,2318,0.1,4378,0.05
intracranial_bleeding_label,106,0.004,156,0.002,73,0.005,189,0.002
GI_bleeding_label,1051,0.04,1926,0.03,1109,0.07,1868,0.02


In [43]:
rs_df.to_csv('./out_table1_AHA_90K_{}_3y.csv'.format(timestr))